In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import e

### Método forward

Se encarga de devolver el resultado de evaluar la función $f_\theta$.
$$f_\theta = W_2 \sigma (W_1 x + b_1)+b_2$$

In [75]:
def forward(W1, W2, b1, b2, x):
    capa1 = W1 @ x + b1
    
    sigmoide = 1/(1+np.exp(-capa1))

    capa2 = W2 @ sigmoide + b2
    return capa2

In [76]:
#Calculo del gradiente numerico
def funcion_objetivo(x, y, W1, W2, b1, b2):
    loss = 0.5*np.mean((forward(W1,W2,b1,b2,x) - y)**2)
    return loss

def numerical_gradient(x, y, W1, W2, b1, b2, eps):
    gradientes = []
    
    gradientes.append(np.zeros(W1.shape))
    gradientes.append(np.zeros(W2.shape))
    gradientes.append(np.zeros(b1.shape))
    gradientes.append(np.zeros(b2.shape))

    for i in range(W1.shape[0]):
        for j in range(W1.shape[1]):
            W1_izq = W1 
            W1_izq[i][j] = W1_izq[i][j] + eps
            W1_der = W1 
            W1_der[i][j] = W1_der[i][j] - eps
            izq_W1 = funcion_objetivo(x, y, W1_izq, W2, b1, b2)
            der_W1 = funcion_objetivo(x, y, W1_der, W2, b1, b2)
            gradientes[0][i][j] = (izq_W1 - der_W1)/(2*eps)
            
    for i in range(W2.shape[0]):
        for j in range(W2.shape[1]):
            W2_izq = W2
            W2_izq[i][j] = W2_izq[i][j] + eps
            W2_der = W2
            W2_der[i][j] = W2_der[i][j] - eps
            izq_W2 = funcion_objetivo(x, y, W1, W2_izq, b1, b2)
            der_W2 = funcion_objetivo(x, y, W1, W2_der, b1, b2)
            gradientes[1][i][j] = (izq_W2 - der_W2)/(2*eps)
    for i in range(b1.shape[0]):
        for j in range(b1.shape[1]):
            b1_izq = b1
            b1_izq[i][j] = b1_izq[i][j] + eps
            b1_der = b1
            b1_der[i][j] = b1_der[i][j] - eps
            izq_b1 = funcion_objetivo(x, y, W1, W2, b1_izq, b2)
            der_b1 = funcion_objetivo(x, y, W1, W2, b1_der, b2)
            gradientes[2][i][j] = (izq_b1 - der_b1)/(2*eps)
    for i in range(b2.shape[0]):
        for j in range(b2.shape[1]):
            b2_izq = b2
            b2_izq[i][j] = b2_izq[i][j] + eps
            b2_der = b2
            b2_der[i][j] = b2_der[i][j] - eps
            izq_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_der)
            der_b2 = funcion_objetivo(x, y, W1, W2, b1, b2_izq)
            gradientes[3][i][j] = (izq_b2 - der_b2)/(2*eps)
    
    return gradientes

In [77]:
# funcion fit y loop de entrenamiento
def fit(x, y, W1, W2, b1, b2, learning_rate, epochs):
    eps = 0.01
    loss_accum = []
    for i in range(epochs):

        gradiente = numerical_gradient(x, y, W1, W2, b1, b2, eps)
        W1 = W1 - learning_rate*gradiente[0]
        W2 = W2 - learning_rate*gradiente[1]
        b1 = b1 - learning_rate*gradiente[2]
        b2 = b2 - learning_rate*gradiente[3]
        
        loss = funcion_objetivo(x, y, W1, W2, b1, b2)
        loss_accum.append(loss)

    theta = [W1, W2, b1, b2]
    return theta, loss_accum

In [78]:
def predict(x, W1, W2, b1, b2):
    y = forward(W1, W2, b1, b2, x)
    return y

### Inicialización de variables W1, b1, W2 y b2

In [79]:
W1 = np.random.random((5, 6))  # matriz 5x6 
b1 = np.random.random((5, 1))  # vector 5x1

W2 = np.random.random((1, 5))  # matriz 1x5
b2 = np.random.random((1, 1))  

### Recolección de datos

Tomamos los datos del archivo y los asignamos desde la posición 0 a la 315 como los de entrenamiento, y desde la 315 hasta el final como los datos de prueba.

In [80]:
df = pd.read_excel("Real estate valuation data set.xlsx")
datos = df.to_numpy()

X = datos[:315, [1,2,3,4,5,6]] 

Y = datos[:315,[7]]

X_test = datos[315:, [1,2,3,4,5,6]]
Y_test = datos[315:,[7]]

### Normalizamos las matrices

In [81]:
# def normalizar(x):
#     for i in range(x.shape[0]):
#         x[i] = (x[i] - x[i].mean())/x[i].std()
#     return x  

# x_norm = normalizar(X)
# x_test_norm = normalizar(X_test)

X_mean = X.mean(axis=1).reshape((X.shape[0],1))
X_std = np.std(X, axis=1).reshape((X.shape[0], 1))

xt_norm_aux = X - X_mean
xt_norm = xt_norm_aux / X_std

X_test_mean = X_test.mean(axis=1).reshape((X_test.shape[0],1))
X_test_std = np.std(X_test, axis=1).reshape((X_test.shape[0], 1))

xt_test_norm_aux = X_test - X_test_mean
xt_test_norm = xt_test_norm_aux / X_test_std


### Entrenamiento de la red

In [82]:
xt_norm = xt_norm.transpose()

res = fit(xt_norm, Y, W1, W2, b1, b2, 0.01, 2000)
theta = res[0]
loss = res[1]

xt_test_norm = xt_test_norm.transpose()

predictions = []

preds = predict(xt_test_norm, theta[0], theta[1], theta[2], theta[3])

print(preds)
print(loss)

# for i in range(X_test.shape[0]): 
#     predictions.append(predict(xt_test_norm[:,i], theta[0], theta[1], theta[2], theta[3]))
#     resta.append(predictions[i] - Y_test[i])
#     print(predictions[i], "- ", Y_test[i], "=", resta[i])

# for i in range(X_test.shape[0]): 
#     predictions.append(predict(Xt_test[:,i], theta[0], theta[1], theta[2], theta[3]))
#     resta.append(predictions[i] - Y_test[i])
#     print(predictions[i], "- ", Y_test[i], "=", resta[i])


[[2.33660733 2.49063407 2.43561945 2.47723787 2.48838891 2.23786726
  2.41204927 2.49680687 2.49602688 2.34008177 2.46536107 2.50935925
  2.435171   2.3583445  2.23787605 2.39135449 2.23237468 2.45084194
  2.50484779 2.40671146 2.48164364 2.32832897 2.45279295 2.48996136
  2.4823503  2.48844749 2.43593558 2.5062769  2.45708559 2.26546204
  2.49693814 2.34012607 2.21245106 2.48965742 2.50484779 2.46487249
  2.30746095 2.28158575 2.30951334 2.3711814  2.47664012 2.49441929
  2.48150013 2.49628811 2.29461156 2.50667936 2.48760282 2.41178594
  2.50456417 2.45137506 2.30248867 2.3331062  2.33108549 2.48009266
  2.30692386 2.48661753 2.48414518 2.49993491 2.48805316 2.47592563
  2.39130329 2.33974296 2.50999968 2.45434311 2.48622493 2.48662239
  2.50232065 2.25239899 2.46332842 2.24028315 2.50704777 2.49694001
  2.31206808 2.30317475 2.50336221 2.47745902 2.32316038 2.47026095
  2.41176347 2.23877685 2.46261332 2.4169845  2.3907243  2.33974296
  2.49850838 2.46594484 2.30774246 2.49680436 2.

In [83]:
# data = {'epochs': range(1000),
#         'loss_acum': loss_acum}

# df = pd.DataFrame(data)

# df.plot(x='epochs', y='loss_acum')

# plt.title('Funcion objetivo a lo largo del entrenamiento')
# plt.xlabel('Epochs')
# plt.ylabel('Loss Acumulated')

# plt.show()